In [ ]:
import os
import sys
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

current_path = os.getcwd()
parrent_path = os.path.abspath(os.path.join(current_path, '..'))
sys.path.append(parrent_path)

from Update_Git import git_add, git_commit, git_push

for i in range (20):
    file_path = os.path.join(current_path, 'Test_7.ipynb')
    git_add(file_path)
    git_commit('Updated test 7')
    git_push('main')

os.environ.pop("OPENAI_API_KEY", None) # Because it loads a key from some place I dont know!
env_path = os.path.join(current_path, ".env")
load_dotenv(dotenv_path=env_path)

True

In [14]:
model = ChatOpenAI(
    model = "gpt-3.5-turbo",
    temperature = 0.5
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly AI assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

memory = ConversationBufferMemory(
    memory_key = "chat_history"
)

chain = memory | prompt | model

msg = {
    "input": "Hello, how are you?",
    return_messages = True
}

response = chain.invoke(
    msg
)

print(response)

SyntaxError: ':' expected after dictionary key (1416063701.py, line 20)